In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
articles = pd.read_json("/fastcampus-data/articles/articels_only_contents.json")
article = articles.copy()
df1 = pd.read_json("/home/user_2/book_articles.json")
df = df1.copy()
# df = df.reset_index() # 인덱스 번호가 이상

# Making book corpus
book의 'name', 'meta_description', 'm_name', 'L_name', 's_name' 정보를 하나의 corpus에 담는다.

In [7]:
# s_name [] 없애기
df['s_name'] = df['s_name'].apply(lambda x: x[0] if type(x) == list else x)

# df에서 name, meta_description, m_name, L_name, s_name 컬럼의 내용을 하나의 컬럼에 담음
df['book_corpus'] = df[['name', 'meta_description', 'm_name', 'L_name', 's_name']].apply(','.join, axis = 1)

df.book_corpus

0      KCD 한방내과 진찰진단 가이드라인,진료에 어려움 있을 때 먼저 찾는 책,전문서적,...
1      유진홍 교수의 이야기로 풀어보는 감염학,이야기로 풀어보는 감염학 1,전문서적,도서,...
2      통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,하이드로다이섹션: 포도당을 이용...
3       의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),없음,전문서적,도서,메디안북
4                 응급의학 실전지침서,꼭 알아야 하는 응급 처치,전문서적,도서,메디안북
                             ...                        
545    Guyton 의학생리학 - Textbook of Medical Physiology,...
546    [세트] 치매의 모든 것, 나는 치매를 다스릴 수 있다,치매 환자를 진료하고 계시는...
547                     아빠가 들려주는 일차진료 소아과학,없음,전문서적,도서,진료
550                             트리거포인트 침치료,없음,전문서적,도서,없음
551       만성질환을 생활습관으로 다스리기,각종 질병 환자를 위한 생활요법,전문서적,도서,생활
Name: book_corpus, Length: 351, dtype: object

책 351권의 corpus와 아티클 3438개의 content_tag_removed의 내용으로 tfidf_metirx를 만든다.

In [5]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()

# none값은 ' '으로 채우기
df['book_corpus'] = df['book_corpus'].fillna(' ')
df['content_tag_removed'] = df['content_tag_removed'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
book_tfidf_matrix = tfidf.fit_transform(df['book_corpus']) # fit은 training data에 사용?
article_tfidf_matrix = tfidf.transform(article['content_tag_removed'])

#Output the shape of tfidf_matrix
print(book_tfidf_matrix.shape)
print(article_tfidf_matrix.shape)

(351, 1485)
(3438, 1485)


In [6]:
# print(book_tfidf_matrix)
# print(article_tfidf_matrix)

# Get Consine similarity using linear_kernel()

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [7]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(book_tfidf_matrix, article_tfidf_matrix).T 

In [8]:
print(pd.DataFrame(cosine_sim).shape)

# 컬럼명을 book name로 바꾸기
df_co_sim = pd.DataFrame(cosine_sim, columns=df.name)

# 인덱스를 article title 으로 바꾸기
df_co_sim = df_co_sim.set_index(article['title'])

print(df_co_sim.shape)
df_co_sim.head(10)

(3438, 351)
(3438, 351)


name,KCD 한방내과 진찰진단 가이드라인,유진홍 교수의 이야기로 풀어보는 감염학,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),응급의학 실전지침서,복부 초음파 정석,알기쉬운 초음파의 이해,초보자를 위한 초음파 첫걸음-전신의 정상 초음파 영상정복!,처음 시작하는 사람들을 위한 경동맥 초음파,최소침습 안면리프팅,...,"극복정신 도서 패키지 [어느 날 400억 원의 빚을 진 남자 & 가자, 어디에도 없었던 방법으로]",생산성 도서 패키지 [생산성 & 일 잘하는 사람은 단순하게 합니다],왕멘즈 방제학 강의,베이징전통의학연구소 후시수 경방의안집,복증기람 복증기람익 세트,"Guyton 의학생리학 - Textbook of Medical Physiology,12/e","[세트] 치매의 모든 것, 나는 치매를 다스릴 수 있다",아빠가 들려주는 일차진료 소아과학,트리거포인트 침치료,만성질환을 생활습관으로 다스리기
title,,,,,,,,,,,,,,,,,,,,,
디스크탈출증 재흡수 케이스,0.000000,0.0,0.000000,0.003020,0.000000,0.0,0.000000,0.492119,0.003375,0.0,...,0.000000,0.000000,0.0,0.007651,0.000000,0.000000,0.000000,0.003642,0.008965,0.000000
여기에 질문하는거 맞나요?,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
뒷목통증 좌측방사통을 주소로 내원한 환자,0.000000,0.0,0.000000,0.004995,0.000000,0.0,0.000000,0.125218,0.005581,0.0,...,0.016611,0.000000,0.0,0.012654,0.000000,0.244523,0.000000,0.033395,0.049289,0.000000
퇴행성 관절염 환자 무릎에 물이 찬 경우.,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
비전형적인 Cauda equina syn.,0.000000,0.0,0.000000,0.000000,0.052365,0.0,0.000000,0.463227,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.012935,0.000000,0.008868,0.000000,0.000000,0.000000
"세무의 정석 3편 : 종합소득세, 소득률.",0.000000,0.0,0.000000,0.081390,0.086833,0.0,0.047853,0.023629,0.000000,0.0,...,0.000000,0.052347,0.0,0.000000,0.000000,0.000000,0.074978,0.000000,0.000000,0.069712
퇴근 직전에 오는 환자들 어떻게 하시는지 궁금합니다~,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
세무의 정석 5편 : 중요한 건 비용처리,0.024761,0.0,0.016538,0.167440,0.178637,0.0,0.017889,0.000000,0.030215,0.0,...,0.038692,0.100961,0.0,0.000000,0.000000,0.000000,0.040268,0.000000,0.000000,0.000000
치료에 반응하지 않는 허리 통증,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.017272,0.492821,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.013761,0.016097,0.009435,0.000000,0.019376,0.000000


In [9]:
df_co_sim.index

Index(['디스크탈출증 재흡수 케이스', '여기에 질문하는거 맞나요?', '뒷목통증 좌측방사통을 주소로 내원한 환자',
       '퇴행성 관절염 환자 무릎에 물이 찬 경우.', '비전형적인 Cauda equina syn.',
       '세무의 정석 3편 : 종합소득세, 소득률.', '퇴근 직전에 오는 환자들 어떻게 하시는지 궁금합니다~',
       '세무의 정석 5편 : 중요한 건 비용처리', '치료에 반응하지 않는 허리 통증', '곧 첫 콘텐츠가 올라옵니다.',
       ...
       '[MZ가 읽는 한의학] 한의학도를 위한 추천 도서- 한의 피부과 편',
       'Ep. 45 한의사가 초음파를 사용하는 건 문제가 안돼요',
       '초음파 케이스: elbow joint effusion (r/o tennis elbow)',
       '시령탕(3-014) 위암 수술 후 여름철 설사, 위장염, 식욕저하, 구갈, 구고에', '피부 진단서 소견서 문의드립니다',
       '[나의 인턴 일지] Ep.04 한의대생, 퇴사를 했다.',
       '초음파 케이스: shoulder impingement synd. (SASD bursitis) (13/4500)',
       '비판적으로 논문 읽기: 통계적 보정에 대해', '[작가가 입사했다] Ep.09 작가가.. 혹사했다..',
       '초음파 케이스: rotator cuff tendon calcification'],
      dtype='object', name='title', length=3438)

In [23]:
scores = list(enumerate(cosine_sim[0])) # cosine_sim[9]: 기사의 인덱스 번호

    
# Sort the movies based on the similarity scores
scores = sorted(scores, key=lambda x: x[1], reverse=True) # 기사와 book의 코싸인 유사도 sort
#     print(sim_scores)

   
scores = scores[1:21]
scores

[(332, 0.29899988770554153),
 (139, 0.09529561450404347),
 (291, 0.049037148385158554),
 (314, 0.04745255990233216),
 (304, 0.046061170055390205),
 (185, 0.04315784276228615),
 (100, 0.025994394401304934),
 (293, 0.025825804345515305),
 (71, 0.022495823095778987),
 (175, 0.02190098930325761),
 (197, 0.021547182728295148),
 (72, 0.020485839058106355),
 (90, 0.01995384832901147),
 (278, 0.018614430206366443),
 (66, 0.018393700807115877),
 (235, 0.01816097600132445),
 (85, 0.017916767229217073),
 (44, 0.017862101803596903),
 (159, 0.014689831354145455),
 (289, 0.014399400868756353)]

In [11]:
#Construct a reverse map of indices and article titles
indices = pd.Series(article.index, index=article['title']).drop_duplicates()

In [21]:
indices.to_frame()[:30]

,0
title,
디스크탈출증 재흡수 케이스,0
여기에 질문하는거 맞나요?,1
뒷목통증 좌측방사통을 주소로 내원한 환자,2
퇴행성 관절염 환자 무릎에 물이 찬 경우.,3
비전형적인 Cauda equina syn.,4
"세무의 정석 3편 : 종합소득세, 소득률.",5
퇴근 직전에 오는 환자들 어떻게 하시는지 궁금합니다~,6
세무의 정석 5편 : 중요한 건 비용처리,7
치료에 반응하지 않는 허리 통증,8


# recomendation: 기사  title을 넣으면 book name 추천

In [15]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]
    print('article index number: ', idx)

    # Get the pairwsie similarity scores 
    sim_scores = list(enumerate(cosine_sim[idx]))
#     print(sim_scores)
    
    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     print(sim_scores)

    # Get the scores of the 20 most similar books
    sim_scores = sim_scores[1:31]
#     print(sim_scores)

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]  # 추천된 것의 index번호
    article_indices_score = [i[1] for i in sim_scores] # cosine_sim score
    df_article_indices_score = pd.DataFrame(article_indices_score) # dataframe으로 만들어서 결과 합치기


    # 추천된 책의 이름과 cosine_sim_score를 합친 dataframe
    recommend_books =pd.DataFrame(df['name'].iloc[article_indices])
    recommend_books['cosine_sim'] = df_article_indices_score.values
    
    # Return the top 20 most similar articles
    return recommend_books


In [16]:
get_recommendations('[MZ가 읽는 한의학] 한의학도를 위한 추천 도서- 한의 피부과 편')

article index number:  3428


,name,cosine_sim
138,한방피부진료레슨,0.279387
189,피부질환의 한의학치료 57증례,0.132677
193,근골격계 통증 및 질환에 대한 임상 시술제요,0.120626
546,"[세트] 치매의 모든 것, 나는 치매를 다스릴 수 있다",0.113276
32,한의통합종양학,0.113034
190,피부질환의 약물과 치료,0.104965
227,한의학 상담,0.099844
175,시니어 피부과학,0.099599
47,수화론(水火論),0.095629
191,의학입문 맥학강의,0.087199


In [40]:
get_recommendations('장골의 AS와 PI 변위란 무엇일까?')

article index number:  3055


,name,cosine_sim
312,침구과 진료매뉴얼,0.161269
11,동씨침법입문 2판,0.160429
163,한 눈에 알 수 있는 류마티스 관절염의 영상진단,0.115223
182,임상에서 자주 쓰이는 근골격계 검진법,0.089446
288,임산부에게 사용할 수 있는 한방처방,0.087267
237,논란의료 - 영리한 그들의 이유 있는 논쟁,0.079371
144,이것만 알면 할 수 있는 치과한방치료,0.067096
94,쉽게 배울 수 있는 근골격계 초음파 입문서,0.059990
323,사암침의 해석과 임상,0.058135
7,초보자를 위한 초음파 첫걸음-전신의 정상 초음파 영상정복!,0.047299


In [20]:
get_recommendations('세무의 정석 7편 : 적격증빙이 중요합니다.')

article index number:  15


,name,cosine_sim
541,생산성 도서 패키지 [생산성 & 일 잘하는 사람은 단순하게 합니다],0.156726
220,한 눈에 알 수 있는 류마티스 관절염의 영상진단,0.131308
4,응급의학 실전지침서,0.125610
367,통장잔고 2배 늘려주는 병의원 절세비법,0.117134
299,혈맥약침,0.115406
386,병의원급 외래에서의 진단과 치료 및 시술제요,0.100230
358,임산부에게 사용할 수 있는 한방처방,0.099450
3,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),0.092830
304,논란의료 - 영리한 그들의 이유 있는 논쟁,0.090451
355,오국통 온병명방,0.088233


In [25]:
df_co_sim[['침구과 진료매뉴얼', '동씨침법입문 2판']][df_co_sim.index == '장골의 AS와 PI 변위란 무엇일까?']

name,침구과 진료매뉴얼,동씨침법입문 2판
title,,
장골의 AS와 PI 변위란 무엇일까?,0.161269,0.160429


In [27]:
print(df.book_corpus[312])
print(df.book_corpus[11])

초음파 유도하 침 시술 가이드북,카드 할인 제외 도서입니다. ,전문서적,도서,한국한의학연구원
동씨침법입문 2판,임상에서 활용할 수 있는 핵심 내용,전문서적,도서,물고기숲


In [39]:
df[['content_tag_removed', 'book_corpus']][df['title']=='장골의 AS와 PI 변위란 무엇일까?']


,content_tag_removed,book_corpus
363,장골의 변위로써 as와 그리고 장단족의 장골의 변위는 장골이 전방회전할 상방으로 위...,"장골의 PI 변위는 없다,짝다리 교정을 위한 골반의 생역학적 이해,전문서적,도서,물고기숲"


In [40]:
df

,name,meta_description,tags,regular_price,$oid,category_id_$oid,m_name,L_name,s_name,id_x,title,score,id_y,subtitle,content_tag_removed,hit,rating,ratings,book_corpus
0,KCD 한방내과 진찰진단 가이드라인,진료에 어려움 있을 때 먼저 찾는 책,[가온해미디어],150000,5d13115e32026c0b35383897,5cf8bbba0098b2225c5dfaa3,전문서적,도서,가온해미디어,2536,군발두통(삼차자율신경 두통),0.250351,2536,KCD 한의임상을 위한 한방내과 진찰진단 치료가이드,상병코드 통증이 동시에 안면부 자율신경계 나타난다 발생하며 여성보다 남성에서 나타나...,728,11,3.0,"KCD 한방내과 진찰진단 가이드라인,진료에 어려움 있을 때 먼저 찾는 책,전문서적,..."
1,유진홍 교수의 이야기로 풀어보는 감염학,이야기로 풀어보는 감염학 1,[군자출판사],30000,5f06a1f3221c8b5f77c8de38,5cf8bbba0098b2225c5dfaa3,전문서적,도서,군자출판사,2536,군발두통(삼차자율신경 두통),0.050401,2536,KCD 한의임상을 위한 한방내과 진찰진단 치료가이드,상병코드 통증이 동시에 안면부 자율신경계 나타난다 발생하며 여성보다 남성에서 나타나...,728,11,3.0,"유진홍 교수의 이야기로 풀어보는 감염학,이야기로 풀어보는 감염학 1,전문서적,도서,..."
2,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,하이드로다이섹션: 포도당을 이용한 신경의 유착박리와 신경주변 환경개선,[메디안북],190000,5e3cb2904267e105dfff13b3,5cf8bbba0098b2225c5dfaa3,전문서적,도서,메디안북,22777,"Dry needling, 어떻게 찌르니? 팔꿈치편",0.072214,22777,Tendinopathy를 다루는 그들(?)의 방법 : 논문리뷰 2,돌려서 넣어주면 낫겠지 수리수리 마수리 나와라 결과는 있습니다 tediothy를 다...,3075,77,4.0,"통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,하이드로다이섹션: 포도당을 이용..."
3,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),없음,[메디안북],130000,5e3d06b64267e105dfff13e0,5cf8bbba0098b2225c5dfaa3,전문서적,도서,메디안북,4178,환자가 한의원에서 넘어졌습니다. 119불러야 할까요?,0.062093,4178,Medistream 온라인 케이스 스터디,가상이지만 pre iter 운영하던 흥미로운 질문이 올라왔습니다 가상이지만 한의원 ...,651,7,3.0,"의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),없음,전문서적,도서,메디안북"
4,응급의학 실전지침서,꼭 알아야 하는 응급 처치,[메디안북],130000,5e4a1d154267e105dfff1d54,5cf8bbba0098b2225c5dfaa3,전문서적,도서,메디안북,4178,환자가 한의원에서 넘어졌습니다. 119불러야 할까요?,0.079849,4178,Medistream 온라인 케이스 스터디,가상이지만 pre iter 운영하던 흥미로운 질문이 올라왔습니다 가상이지만 한의원 ...,651,7,3.0,"응급의학 실전지침서,꼭 알아야 하는 응급 처치,전문서적,도서,메디안북"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,"Guyton 의학생리학 - Textbook of Medical Physiology,...","Textbook of Medical Physiology,12/e",[],88000,6045d817643a8a0019f2c187,5cf8bbba0098b2225c5dfaa3,전문서적,도서,원서,2617,어깨 체외충격파에 대해 간단하게 이해하기,0.055752,2617,"Medistream 어깨 진단, 검진 30편",eswt의 치료원리 충격입니다 실제로 받아보면 아픕니다 고가입니다 받는데 합니다 이...,1350,18,3.0,"Guyton 의학생리학 - Textbook of Medical Physiology,..."
546,"[세트] 치매의 모든 것, 나는 치매를 다스릴 수 있다",치매 환자를 진료하고 계시는 분들께 추천합니다.,[],56000,608fb4abc1908b001a6407c1,5cf8bbba0098b2225c5dfaa3,기타,도서,세트,4182,치매의 원인과 진단,0.155632,4182,KCD 한의임상을 위한 한방내과 진찰진단 치료가이드,상병코드 치매인지증란 후천적인 기질적 장애로 정상적으로 발달되었던 지능이 확연하게 ...,263,7,3.0,"[세트] 치매의 모든 것, 나는 치매를 다스릴 수 있다,치매 환자를 진료하고 계시는..."
547,아빠가 들려주는 일차진료 소아과학,없음,[],100000,60f7d00a09c63a06ed1476c2,5cf8bbba0098b2225c5dfaa3,전문서적,도서,진료,5651,"[신향화의 핵심 소아과 상식] 10. 장염(설사,구토)의 관리법",0.071196,5651,더 정확한 한 마디를 위한 핵심 소아과 상식,로타바이러스 예방접종에 이어서 아이가 증상이 어떻게 대처해야 살펴보겠습니다 아이가 ...,768,11,3.0,"아빠가 들려주는 일차진료 소아과학,없음,전문서적,도서,진료"
550,트리거포인트 침치료,없음,[],65000,617a3aacbc63410023471ab6,5cf8bbba0098b2225c5dfaa3,전문서적,도서,없음,20211,상부승모근 뭉침 침치료 어떻게하나요?,0.162853,20211,None,어디에 어떻게 침치료 해야하는건가요,621,-1,2.5,"트리거포인트 침치료,없음,전문서적,도서,없음"
